In [1]:
import numpy as np
import numpy.random as npr
from scipy import sparse
from scipy.spatial.distance import cosine
from time import time

**Storage in the network model**

In [2]:
def get_one(n):
    """Return array of ones of length n."""
    return np.ones(n)

def get_weights_rand(shape, prob):
    """Return a weights matrix with prob amount of random connections activated.
    
    Args:
        shape: (int) number of neurons in network
        prob: (float) fraction of connections to be activated
    
    Returns:
        weights: (numpy matrix) connectivity weight matrix
    """
    l = int(shape*prob)
    rows = npr.choice(range(shape), l)
    cols = npr.choice(range(shape), l)
    weights = sparse.csr_matrix((np.ones(l), (rows, cols)), shape=(shape,shape))
    weights.setdiag(0)
    return weights

def get_patterns(activity, num_neurons, num_patterns):
    """Return a matrix where each row is a pattern with activity level activity.
    
    Args:
        activity: (float) avg activity level per pattern
        num_neurons: (int) num neurons in network
        num_patterns: (int) num patterns to generate
    
    Returns:
        patterns: (numpy matrix) set of patterns shape=(num_patterns, num_neurons)
    """
#     l = int(num_neurons*activity)
#     rows = npr.choice(range(num_patterns), l)
#     cols = npr.choice(range(num_neurons), l)
#     patterns = sparse.csr_matrix((np.ones(l), (rows, cols)), shape=(num_patterns, num_neurons))
    return npr.binomial(1, activity, size=(num_patterns, num_neurons))

def get_syn_weights(patterns):
    """Return synaptic weight matrix.
    
    Args:
        patterns: (numpy matrix) set of patterns, shape=(num_patterns, num_neurons)
        
    Returns:
        syn: (numpy matrix) synaptic weight matrix, shape=(num_neurons, num_neurons)
    """
    num_patterns, num_neurons = patterns.shape
    row_ind = []
    col_ind = []
    for i in range(num_patterns):
        p = sparse.csr_matrix(patterns[i, :])
        mat = p * p.T
        rows, cols = np.nonzero(mat)
        row_ind = np.concatenate([row_ind, rows])
        col_ind = np.concatenate([col_ind, cols])
    o = np.ones(len(row_ind))
    return sparse.csr_matrix((o, (row_ind, col_ind)), shape=(num_neurons, num_neurons))

# everything is decreased by 1 order of magnitude from paper

n = 30000                  # number of neurons in network; CA3 is 330,000 (or double)
p = .03                   # connection probability for weight matrix
m = 5                   # number of patterns to store
f = 0.005                  # average activity level of each pattern

now = time()
W = get_weights_rand(n, p) # connectivity weight matrix; are neurons connected?
print('made W in', time() - now)
now = time()
M = get_patterns(f, n, m)  # patterns
print('made M in', time() - now)
now = time()
J = get_syn_weights(M)     # synaptic weight matrix J; how much are neurons connected?
print('made J in', time() - now)
now = time()

W.shape, M.shape, J.shape

/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


made W in 0.3764629364013672
made M in 0.002683877944946289
made J in 0.0037589073181152344


((30000, 30000), (5, 30000), (30000, 30000))

In [ ]:
# TOO SLOW!!
print('W symmetric?', W!=W.T)
print('should be no')
print('J symmetric?', J!=J.T)
print('should be yes')

**Recall in the network model**

In [4]:
def get_degraded_pattern(pattern, valid, spurious):
    """Return degraded pattern for input to simulation to do pattern completion.
    
    Args:
        pattern: (array) pattern to degrade, 0s or 1s, shape=(num_neurons)
        valid: (float) fraction of valid firing neurons
        spurious: (float) fraction of spurious firing neurons
    
    Return:
        deg_pat: (array) degraded pattern, 0s or 1s, shape=(num_neurons)
    """
    print('pattern of len {} with {} fires'.format(len(pattern), sum(pattern)))
    fires = np.nonzero(pattern)[0]
    deg_fires = npr.choice(fires, size=int(valid*len(fires)), replace=False)

    nonfires = np.where(pattern == 0)[0]
    spur_fires = npr.choice(nonfires, size=int(spurious*len(nonfires)), replace=False)
    
    all_fires = np.concatenate((deg_fires, spur_fires), axis=0)

    deg_pat = np.zeros(shape=(len(pattern),))
    deg_pat[all_fires] = 1
    
    return deg_pat


def simulate(in_pattern, out_pattern, con_mat, syn_mat, g0, g1, cycles=10, pprint=False):
    """Return final matrix of updates.
    
    Args:
        in_pattern: (array) degraded pattern to start
        out_pattern: (array) original pattern to attempt to recall
        con_mat: (matrix) connectivity matrix
        syn_mat: (matrix) synaptic weights matrix
        g0: (float) firing threshold
        g1: (float) inhibition factor
        cycles: (int) num times to run iteration
    
    Return:
        curr_mat: (matrix) end state matrix
    """
    n = len(in_pattern)
    W_J = con_mat.multiply(syn_mat)
    state = in_pattern
    if pprint:
        print('{:<10} {:<10} {:<10}'.format('steps', 'correl', 'num fires'))
    for i in range(cycles):
        if pprint:
            pass
#             print('{:<10} {:<10.2f} {:<10}'.format(i, get_correlation(state, out_pattern), sum(state)))
        _inter = W_J.dot(state)
        h = (1/n) * _inter
        condition = h - ( (1/n) * g1 * np.sum(state) )
        state = condition > g0
    return state

# def get_correlation(p1, p2):
#     p1_ = np.repeat(p1 - (sum(p1)/len(p1)), len(p1))
#     p2_ = np.repeat(p2 - (sum(p2)/len(p2)), len(p2))
#     return 1 - cosine(p1_, p2_)

b_vald = .5  # fraction of valid firing neurons in patterns
b_spur = 0.001  # fraction of spurious firing neurons in patterns


now = time()
godpat = M[0, :]
badpat = get_degraded_pattern(godpat, b_vald, b_spur)
print('made degraded pattern in', time() - now)

g_0 = 7 * 10**-6
g_1 = 0.0
res = simulate(badpat, godpat, W, J, g_0, g_1, pprint=False)
# should see a correlation between original and degraded patterns of 0.71

pattern of len 30000 with 160 fires
made degraded pattern in 0.013550996780395508


In [238]:
n = 1000                  # number of neurons in network; CA3 is 330,000 (or double)
p = .7                   # connection probability for weight matrix
m = 5                   # number of patterns to store
f = 0.005                  # average activity level of each pattern

b_vald = .5  # fraction of valid firing neurons in patterns
b_spur = 0.00  # fraction of spurious firing neurons in patterns

g_0 = 7 * 10**-6
g_1 = 0.0

p_range = [.1, .3, .5, .7, .9]
m_range = [1, 3, 5, 7, 8]

res_mat = np.zeros(shape=(5,5))

for i, p in enumerate(p_range):
    for j, m in enumerate(m_range):
        W = get_weights_rand(n, p) # connectivity weight matrix; are neurons connected?
        M = get_patterns(f, n, m)  # patterns
        J = get_syn_weights(M)     # synaptic weight matrix J; how much are neurons connected?
        
        godpat = M[0, :]
        badpat = get_degraded_pattern(godpat, b_vald, b_spur)

        res = simulate(badpat, godpat, W, J, g_0, g_1)
        res_mat[i, j] = res

res_mat

pattern of len 1000 with 6 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 5 fires


/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


pattern of len 1000 with 6 fires
pattern of len 1000 with 2 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 5 fires
pattern of len 1000 with 3 fires
pattern of len 1000 with 8 fires
pattern of len 1000 with 5 fires
pattern of len 1000 with 5 fires
pattern of len 1000 with 6 fires
pattern of len 1000 with 10 fires
pattern of len 1000 with 9 fires
pattern of len 1000 with 3 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 6 fires
pattern of len 1000 with 4 fires
pattern of len 1000 with 6 fires
pattern of len 1000 with 9 fires


array([[        nan,         nan,         nan,         nan,         nan],
       [ 0.86559098,  0.44631737,         nan,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  0.57677205],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  0.81567639,  1.        ,  1.        ]])

In [19]:
x = np.array([[0,1,0], [0,0,0], [1,1,0]])
print(x)
rows, cols = np.nonzero(x)
y = []
y = np.concatenate([y, rows])
print(y)

[[0 1 0]
 [0 0 0]
 [1 1 0]]
[ 0.  2.  2.]
